# ECDSA Nonce Attack

In the case of deterministic ECDSA, the nonce $k$ is generated based on the private key and the message being signed. If there is a flaw in the implementation that causes $k$ to be predictable or reused, it becomes possible to derive the private key from multiple signatures.

The attack works as follows:

1. Nonce Reuse: If the same nonce $k$ is used to sign two different messages, the private key can be computed directly.

2. Partial Nonce Leakage: If the nonce $k$ is partially leaked or follows a predictable pattern, statistical analysis over multiple signatures can be used to reconstruct the private key.

### How the Attack Works

Assume you have two signatures $(r_1, s_1)$ and $(r_2, s_2)$ for messages $m_1$ and $m_2$ respectively, signed with the same nonce $k$.

The ECDSA signature generation process is:

Compute $r = (k \cdot G)_x \mod n$

Compute $s = k^{-1}(H(m) + r \cdot d) \mod n$

Where:

$G$ is the base point of the elliptic curve.

$d$ is the private key.

$H(m)$ is the hash of the message.

$k$ is the nonce.

Given $r1 = r2$ (same nonce used):

$s_1 = k^{-1}(H(m_1) + r \cdot d) \mod n$
$s_2 = k^{-1}(H(m_2) + r \cdot d) \mod n$

We can rewrite these as:
$k = \frac{H(m_1) + r \cdot d}{s_1} \mod n$
$k = \frac{H(m_2) + r \cdot d}{s_2} \mod n$

Equating the two expressions for $k$:

$\frac{H(m_1) + r \cdot d}{s_1} = \frac{H(m_2) + r \cdot d}{s_2}$

Rearranging to solve for $d$:

$d = \frac{H(m_1) \cdot s_2 - H(m_2) \cdot s_1}{r \cdot (s_1 - s_2)} \mod n$

In [24]:
import jupyprint as jp
from ecdsa import SigningKey, SECP256k1
import hashlib
from ecdsa.numbertheory import inverse_mod
from ecdsa.util import number_to_string

# Generate a new private key using the SECP256k1 curve
private_key = SigningKey.generate(curve=SECP256k1)
public_key = private_key.verifying_key
order = SECP256k1.order

print("Private key:", private_key.to_string().hex())
print("Public key:", public_key.to_string().hex())

# Fixed deterministic nonce
k = 1234523534  # Small value for demonstration; in practice, it would be a securely generated number

# Function to hash a message
def hash_message(message):
    return int(hashlib.sha256(message).hexdigest(), 16)

# Function to sign a message with a fixed nonce
def sign_message_with_fixed_nonce(private_key, message, k):
    hash = hash_message(message)
    k_inverse = pow(k, -1, order)
    r = (k * SECP256k1.generator).x() % order
    s = (k_inverse * (hash + r * private_key.privkey.secret_multiplier)) % order
    return r, s

# Sign two messages with the same deterministic nonce
message1 = b"Message 1"
message2 = b"Message 2"

r1, s1 = sign_message_with_fixed_nonce(private_key, message1, k)
r2, s2 = sign_message_with_fixed_nonce(private_key, message2, k)

print(f"Signature 1: r = {r1}, s = {s1}")
print(f"Signature 2: r = {r2}, s = {s2}")

h1 = hash_message(message1)
h2 = hash_message(message2)

# Ensure r values are the same, indicating nonce reuse
if r1 != r2:
    raise ValueError("Nonces are different, attack not applicable")

# Calculate the private key
r = r1
numerator = (h1 * s2 - h2 * s1) % order
denominator = (r * (s1 - s2)) % order
denominator_inv = inverse_mod(denominator, order)
private_key_recovered = (numerator * denominator_inv) % order

print("Recovered private key:", hex(private_key_recovered))
print("Actual private key:", private_key.to_string().hex())

# Create a SigningKey from the recovered private key
sk_recovered = SigningKey.from_secret_exponent(private_key_recovered, curve=SECP256k1)
print("Recovered key matches the original:", sk_recovered.to_string().hex() == private_key.to_string().hex())

# Verify the recovered private key can sign a new message
new_message = b"New message"
new_signature = sk_recovered.sign(new_message)
print("New signature:", new_signature.hex())

# Verify the new signature with the original public key
try:
    assert public_key.verify(new_signature, new_message)
    print("The new signature is valid.")
except Exception as e:
    print("The new signature is invalid.", e)


Private key: eafd830c35746e71ba84ed48ce7abae51c074b764bc7a4806f64ed0bf7ef12c8
Public key: deb0aa501e5fe02b6ba656e4f2e754b91ef55adcda3e18da5c1daedcb0e7225dce0e8f737d0f08500f14586bd5a85025a33f82944c464a6bea02036942fd5cdd
Signature 1: r = 111458020428249003069305291817160846463593890304406410981730868625359210756736, s = 107868655616028798820312505238264530331467692059802992574186917116798630338734
Signature 2: r = 111458020428249003069305291817160846463593890304406410981730868625359210756736, s = 115415888967636740926999122226077969081537178177634416660045343807561551294492
Recovered private key: 0xeafd830c35746e71ba84ed48ce7abae51c074b764bc7a4806f64ed0bf7ef12c8
Actual private key: eafd830c35746e71ba84ed48ce7abae51c074b764bc7a4806f64ed0bf7ef12c8
Recovered key matches the original: True
New signature: 0f77101d19ce8b65633255e51357c5476cd590c09eff79a6d0cfacd6c58682520c16d4fe75fb4b763a0d921308be57a130fc56506a01265a59aade7f7f743a1a
The new signature is valid.


## Why does this attack work?

### The Basics of ECDSA

Elliptic Curve Digital Signature Algorithm (ECDSA) is a method used to create digital signatures. Digital signatures help verify that a message has not been tampered with and confirm the identity of the sender.

Here's a simplified version of how ECDSA works:

1. Key Pair Generation:

    * Private Key: A randomly chosen large number (let's call it $d$).

    * Public Key: A point on the elliptic curve derived from the private key (let's call it $Q$).

2. Signing a Message:

    * Hash the Message: Compute a hash of the message ($h$).

    * Generate a Random Number: Generate a random number ($k$) called a nonce.

    * Compute Signature Components:

        * $r = (k \cdot G)_x \mod n$
        * $s = k^{-1} (h + r \cdot d) \mod n$
        
    * The signature is the pair $(r, s)$

3. Verifying a Signature:

* Using the public key and the signature, anyone can verify the authenticity of the message.

### The Importance of the Nonce

The nonce $k$ is a critical part of the signing process. It needs to be unique and random for each signature. If the same nonce is used for two different signatures, it can lead to a security breach. This is because $k$ helps ensure that each signature is unique, even if the same message is signed multiple times.

The attack works because reusing the same nonce $k$ in multiple signatures allows an attacker to extract the private key $d$. Here's how:

### Step-by-Step Explanation

1. Two Messages Signed with the Same Nonce:

    * Suppose we have two messages $m_1$ and $m_2$ signed with the same private key $d$ and the same nonce $k$.
    * The signatures for these messages are:
        * Signature 1: $(r, s_1)$ for message $m_1$
        * Signature 2: $(r, s_2)$ for message $m_2$

2. ECDSA Signature Equations:
    * For message $m_1$

        $s_1 = k^{-1} (h_1 + r \cdot d) \mod n$
    * For message $m_2$:

        $s_2 = k^{-1} (h_2 + r \cdot d) \mod n$

    * Here, $h_1$ and $h_2$ are hashes of $m_1$ and $m_2$, respectively.

3. Solving for the Nonce:

    * Rearrange the equations to isolate $k$:

        $k = (h_1 + r \cdot d) \cdot s_1^{-1} \mod n$
        
        $k = (h_2 + r \cdot d) \cdot s_2^{-1} \mod n$
    * Since $k$ is the same in both equations, set them equal to each other:

        $(h_1 + r \cdot d) \cdot s_1^{-1} \equiv (h_2 + r \cdot d) \cdot s_2^{-1} \mod n$

4. Extracting the Private Key:

    * With a bit of algebra, you can solve for the private key $d$:
    
        $d = \frac{h_1 \cdot s_2 - h_2 \cdot s_1}{r \cdot (s_1 - s_2)} \mod n$


This equation shows that if an attacker knows the signatures $(r, s_1)$ and $(r, s_2)$ for two different messages signed with the same nonce, they can compute the private key $d$.